<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Transforming Excel Analysis into pandas Data Models</h1>
<h1>Basic Excel Spreadsheets</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

import openpyxl

import watermark

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

numpy      1.18.1
pandas     1.0.1
openpyxl   3.0.5
json       2.0.9
autopep8   1.5
matplotlib 3.1.3
watermark  2.0.2
Thu Sep 03 2020 

CPython 3.7.3
IPython 6.2.1

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 19.6.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit
Git hash   : db6f805bfd2f0754fac849822010bc72ecbfeb20


Load default figure style

In [3]:
plt.style.use('./d4sci.mplstyle')

## Create a new empty workbook

In [4]:
book = openpyxl.Workbook()

Workbooks are always created with at least one sheet

In [5]:
book.sheetnames

['Sheet']

We can index it directly by name:

In [6]:
sheet = book['Sheet']
print(sheet)

<Worksheet "Sheet">


Or simply get the currently active one

In [7]:
sheet = book.active
print(sheet)

<Worksheet "Sheet">


And to rename it we can doo

In [8]:
sheet.title = 'My Data'

And now we see that the name has changed, as expected

In [9]:
book.sheetnames

['My Data']

The sheet is currently empty

In [10]:
sheet.dimensions

'A1:A1'

Let's add some data

In [11]:
sheet['A3'] = "Hello"
sheet['B3'] = "World"

Now we see that we have a few more rows/columns

In [12]:
sheet.dimensions

'A3:B3'

In [13]:
rows = np.arange(20).reshape(10, 2)

In [14]:
rows

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11],
       [12, 13],
       [14, 15],
       [16, 17],
       [18, 19]])

In [15]:
for row in rows:
    sheet.append(list(row)) # Rows have to be lists or tuples

In [16]:
sheet.dimensions

'A3:B13'

And finally we can save our new workbooks

In [17]:
book.save('data/Simple.xlsx')

In [18]:
!open data/Simple.xlsx

## Load an existing Workbook

In [19]:
book = openpyxl.load_workbook('data/Simple.xlsx')

List all the available worksheets

In [20]:
book.sheetnames

['My Data']

Get the worksheet by name

In [21]:
sheet = book['My Data']

Print some statistics

In [22]:
print(sheet.dimensions)
print("Min row: %u Max row: %u" % (sheet.min_row, sheet.max_row))
print("Min col: %u Max col: %u" % (sheet.min_column, sheet.max_column))

A3:B13
Min row: 3 Max row: 13
Min col: 1 Max col: 2


Print all the values:

In [23]:
for c1, c2 in sheet[sheet.dimensions]:
    print(c1.value, c2.value)

Hello World
0 1
2 3
4 5
6 7
8 9
10 11
12 13
14 15
16 17
18 19


## Add a new sheet

In [24]:
sheet = book.create_sheet(title="My analysis", index=0)

In [25]:
book.sheetnames

['My analysis', 'My Data']

In [26]:
sheet

<Worksheet "My analysis">

In [27]:
sheet["B4"] = "The best results"
sheet["B5"] = "guaranteed!"

and the file again:

In [28]:
book.save("data/Simple2.xlsx")

In [29]:
!open data/Simple2.xlsx

## Remove sheets from an existing file

In [30]:
book = openpyxl.load_workbook("data/movies.xlsx")

In [31]:
book.sheetnames

['1900s', '2000s', '2010s', '3000s']

In [32]:
book.remove(book["3000s"])

In [33]:
book.sheetnames

['1900s', '2000s', '2010s']

In [34]:
book.save('data/movies2.xlsx')

In [35]:
!open data/movies2.xlsx

## Extract formulas 

In [36]:
book = openpyxl.load_workbook('data/excel-mortgage-calculator.xlsx')

/Users/bgoncalves/anaconda3/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [37]:
book.sheetnames

['Excel Amortization Schedule']

In [38]:
sheet = book.active

In [39]:
sheet.dimensions

'B3:K376'

The computed values start at B17, so we print that row plus the previous one for the headers.

In [40]:
for row in sheet["B16:K17"]:
    for cell in row:
        print("%s%s: '%s'" % (cell.column_letter, cell.row, cell.value))
    print("")

B16: 'PMT NO'
C16: 'PAYMENT DATE'
D16: 'BEGINNING BALANCE'
E16: 'SCHEDULED PAYMENT'
F16: 'EXTRA PAYMENT'
G16: 'TOTAL PAYMENT'
H16: 'PRINCIPAL'
I16: 'INTEREST'
J16: 'ENDING BALANCE'
K16: 'CUMULATIVE INTEREST'

B17: '=IF(LoanIsGood,IF(ROW()-ROW(PaymentSchedule[[#Headers],[PMT NO]])>ScheduledNumberOfPayments,"",ROW()-ROW(PaymentSchedule[[#Headers],[PMT NO]])),"")'
C17: '=IF(PaymentSchedule[[#This Row],[PMT NO]]<>"",EOMONTH(LoanStartDate,ROW(PaymentSchedule[[#This Row],[PMT NO]])-ROW(PaymentSchedule[[#Headers],[PMT NO]])-2)+DAY(LoanStartDate),"")'
D17: '=IF(PaymentSchedule[[#This Row],[PMT NO]]<>"",IF(ROW()-ROW(PaymentSchedule[[#Headers],[BEGINNING BALANCE]])=1,LoanAmount,INDEX(PaymentSchedule[ENDING BALANCE],ROW()-ROW(PaymentSchedule[[#Headers],[BEGINNING BALANCE]])-1)),"")'
E17: '=IF(PaymentSchedule[[#This Row],[PMT NO]]<>"",ScheduledPayment,"")'
F17: '=IF(PaymentSchedule[[#This Row],[PMT NO]]<>"",IF(PaymentSchedule[[#This Row],[SCHEDULED PAYMENT]]+ExtraPayments<PaymentSchedule[[#This Ro

If instead of opened the file with data_only=True, we would get the numerical values instead

In [41]:
book = openpyxl.load_workbook('data/excel-mortgage-calculator.xlsx', data_only=True)
sheet = book.active

for row in sheet["B16:K17"]:
    for cell in row:
        print("%s%s: '%s'" % (cell.column_letter, cell.row, cell.value))
    print("")

B16: 'PMT NO'
C16: 'PAYMENT DATE'
D16: 'BEGINNING BALANCE'
E16: 'SCHEDULED PAYMENT'
F16: 'EXTRA PAYMENT'
G16: 'TOTAL PAYMENT'
H16: 'PRINCIPAL'
I16: 'INTEREST'
J16: 'ENDING BALANCE'
K16: 'CUMULATIVE INTEREST'

B17: '1'
C17: '2020-09-01 00:00:00'
D17: '200000'
E17: '1073.6432460242781'
F17: '100'
G17: '1173.6432460242781'
H17: '340.30991269094477'
I17: '833.3333333333334'
J17: '199659.69008730905'
K17: '833.3333333333334'



/Users/bgoncalves/anaconda3/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


<div style="width: 100%; overflow: hidden;">
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</div>